<center>
    
<h1>An example of the Gilepsie Algorithm: the Hok/Sok mechanism.</h1>
</center>

We propose in this project to model a simplify the Hok/sok system. We will propose to code a stochastic simulator that will represent the production of both Hok and Sok mRNAs and Hok proteins.

## The Hok/sok system

Text adapted from from wikipedia (https://en.wikipedia.org/wiki/Hok/sok_system):

<i>
The hok/sok system is a postsegregational killing mechanism employed by the R1 plasmid in Escherichia coli. […] The hok/sok system involves two genes:
- hok, host killing : RNA with a long lifetime, its protein is toxic
- sok, suppression of killing : RNA with a short lifetime, RNA antitoxin

When E. coli undergoes cell division, the two daughter cells inherit the long-lived hok toxin from the parent cell. Due to the short half-life of the sok antitoxin, daughter cells inherit only small amounts and it quickly degrades.

If a daughter cell has inherited the R1 plasmid, it has inherited the sok gene and a strong promoter which brings about high levels of transcription. So much so that in an R1-positive cell, Sok transcript exists in considerable molar excess over Hok mRNA. Sok RNA then indirectly inhibits the translation of hok by inhibiting mok translation. There is a complementary region where sok transcript binds hok mRNA directly (pictured). Instead, sok RNA regulates the translation of the mok open reading frame, which nearly entirely overlaps that of hok. It is this translation-coupling which effectively allows sok RNA to repress the translation of hok mRNA.

The sok transcript forms a duplex with the leader region of hok mRNA and this is recognized by RNase III and degraded. The cleavage products are very unstable and soon decay.

With plasmid || When the plasmid leaves
-||-
<img src="./figures/wikipedia.svg" alt="Transcription" style="width: 400px;"/> &nbsp;&nbsp;&nbsp;||&nbsp;&nbsp;&nbsp;  <img src="./figures/wikipedia1.svg" alt="TwoStageModel1.png" style="width: 400px;"/>

(image inspired by [this wikipedia diagram](https://en.wikipedia.org/wiki/Hok/sok_system#/media/File:Hok_sok_system_R1_plasmid_present.gif) )

</i>

***Some useful data***:

Given parameters:
- the reaction rate of the mRNA hok and sok is  $3 \cdot 10^2 M^{-1} s^{-1}$ (Franch et al., 1997)
- the typical E. Coli volume $1\,\mu m^3$

Parameter you will have to search in the litterature:
- hok mRNA half-life
- sok RNA half-life

Other information that are not easy to find in the literature. To begin with, we can propose that:
- rate of hok mRNA creation $0.3\,min^{-1}$
- rate of sok RNA creation $3\,min^{-1}$
- rate of Hok protein creation per mRNA $0.3\,min^{-1}$
- rate of Hok protein proteolysis $0.01\,min^{-1}$







# Let's play with random numbers

As we will do stochastic simulations, we will need to play with random numbers. For that, we will import the scientific library **numpy** and the plotling library ***pyplot*** (we will use ***plt*** as a shortcut for ***pyplot***)

In [1]:
import numpy
import matplotlib.pyplot as plt
# The next statement ensure that the figure are plot in the notebook
%matplotlib inline 

There is a submodule of numpy that deals with random numbers: ***numpy.random***. From this, you have access to a series usual random distribution: 

- `numpy.random.random()` : returns a random float in $[0,1[$
- `numpy.random.randint(N)` : returns a random integer between 0 and N (without including N)
- `numpy.random.exponential(scale)` : returns a float that follow an exponentially distributed random distribution of parameter $1/scale$. ***Caution:*** for some reason, it is $1/scale$ and not $scale$. 
- etc.

Let's try:

In [2]:
for i in range(10):
    print("A random integer: ",numpy.random.randint(10)) # prints a random integer between 0 and 10 (without including 10)

A random integer:  4
A random integer:  0
A random integer:  3
A random integer:  0
A random integer:  2
A random integer:  0
A random integer:  5
A random integer:  9
A random integer:  3
A random integer:  9



For each of these function, you can specify a size, in which case the function will return an ***array*** of the specify size. For instance, `numpy.random.randint(10,size=100)` will return a array of 100 integers randomly chosen between 0 and 10 (without including 10).

***Exercise*** : Create an array of 1000 elements that follow a exponential random distribution of parameter 0.1 and plot its histogram.



In [ ]:
#### TO COMPLETE
distribParam = 0.1
randomArray = ???????????????????????

####

plt.hist(randomArray,normed=True,bins=30,alpha=.5,label='Random') # This statement plots the histogram
X = numpy.linspace(0,70,100)
plt.plot(X,distribParam * numpy.exp(-distribParam*X),label='Distribution') # This statement plots the distribution
plt.legend()

One important random distribution is missing in numpy.random . It is the *weighted choice*.


Example: Let's imagine you have a the biased dice, such as you have:
- 1/2 chance to obtain a 1
- 1/10 chances to obtain a 2
- 1/10 chances to obtain a 3
- 1/10 chances to obtain a 4
- 1/10 chances to obtain a 5
- 1/10 chances to obtain a 6
<img src="https://upload.wikimedia.org/wikipedia/commons/a/a5/6sided_dice.jpg" alt="Dice" style="width: 200px;"/>
<div style="text-align: right">[image from wikipedia](https://upload.wikimedia.org/wikipedia/commons/a/a5/6sided_dice.jpg)</div>

(where if you had a fair dice, the chances will be 1/6 for each of them).

How to draw a sample from such random variable ? The function to draw it is not given in **numpy.random**. We will need it in the project, so it will be necessary to create this function ourself. Even if it is not complicated to code, I give it directly in the next cell.

**Note**: this function do not specifically expects weights as a list of probabilities that sums to 1. If the sum of all the weights is not 1, it automatically normalized them. The function also returns the sum of all the weights (it will be useful later). 

In [ ]:
def randomWeightedChoice(weights,size=1):
    """
    Randomly choice an index accorging to the given weights.
    - weights: numpy 1D array like
            The probability weights for each choice (if the sum of all weights 
            is not 1, we scale it to 1)
    - size: int
            The size of the sample
    
    Returns:
    - index: int or 1DArray
        The indexes of the choice (if size==1, returns an interger)
    - summ: the sum of all the weights (before normalizing)
    """
    bins = numpy.add.accumulate(weights,dtype=float)
    summ = bins[-1]
    
    if size==1:
        return  int(numpy.searchsorted(bins,summ*numpy.random.random_sample())),summ
    else:
        result = numpy.zeros(size,dtype=int)
        samples = summ*numpy.random.random_sample(size=size)

        for i,s in enumerate(samples):
            result[i] = int(numpy.searchsorted(bins,s))
            
        return result,summ

This function will be VERY USEFULL in the remaining of the project.

**Exercice:** Let's check that this function works as expected. We will try the `randomWeightedChoice` on the previous example of the bias dice. Sample a large array with  `randomWeightedChoice` with the weights corresponding to the bias dice example. Check that the obtained histogram corresponds to the expected distribution.

In [ ]:
diceWeights = numpy.array([0.5,0.1,0.1,0.1,0.1,0.1])


#### TO COMPLETE
randomArray = ???????????????????????

plt.hist(????????????????????,normed=True,bins=numpy.arange(7),
                     alpha=.5,label='Random Choice') # This statement plots the histogram
plt.plot(?????????????????????,label='Distribution') # This statement plots the distribution
plt.legend()

####


# Gillepsie algorithm's main tool: exponential distributions

This section contains the only maths that will be usefull to perform our simulations.

The exponential distribution is a continuous distribution (i.e. it represents floats) and it often used to represent periods of time. Its probability density function is given by $$f(x)=a e^{-a\cdot x}.$$

Typically the parameter $a$ of the distribution given the **rate** at which happen the event. 
 ***
**Example:** For instance, let's model the mRNA degradation with a exponentially random variable, this mRNA degrates on average in 10 minutes, the parameter of the exponential dsitribution will then be $0.1\,min^{-1}$.
***

**Note:** In Biology a more common parameter would be the half-life time. The rate $a$ of an event and the half-time $t_{1/2}$ covers the same concept. They are linked by the formula:
$$t_{1/2}=\frac{\log 2}{a}$$

In [ ]:
rateDegradation = 0.1 # min^-1
print("The mRNA will degrade in %.3f min."%numpy.random.exponential(1/rateDegradation)) # caution, it is 1/param

A very common hypothesis in stochastic modelling of biological event is to represent every event (encounter between molecules, molecule degradations, molecule elongations, etc.) as randomly exponentially distributed variables.There is three main reason to do so:

- It is often quite close to the reality (for instance, the degradation of an mRNA).
- It is much easier to get analytical formula from such models
- <u>It is much simpler to simulate such models</u>

Why is it much simpler to simulate by only considering exponentially distributed random variables? Because there is a very usefull trick when using only exponentially distributed random variables that we can use:

***
**Example**: Let's imagine we have two mRNAs, one is degrading at a rate $0.1\,min^{-1}$, the other at a rate of $0.3\,min^{-1}$. We choose to model each of these event with exponentially distributed variables.
1. The time WHEN the first of the two degradations happend follows a exponential distribution of parameter $(0.1+0.3)\,min^{-1}$
2. WHICH of the degradation events happen first is given by a random weigthed choice with weights $$\left(\frac{0.1}{0.1+0.3},\frac{0.3}{0.1+0.3}\right).$$ (that means there is a probability of $\frac{0.1}{0.1+0.3}$ that the first mRNA degrades at first and $\frac{0.3}{0.1+0.3}$ that the second mRNA degrades at first.


***    

This idea is at the core of the Gillespie Algorithm.

Let's check the first part (the "WHEN part") of the property : 

***
**Exercise:** 
- create an array `Xarray` of 1000 elements following an exp. distribution of param 0.1
- create an array `Yarray` of 1000 elements following an exp. distribution of param 0.3
- create an array `minXYarray` of 1000 elements such as `minXYarray[i]` is the minimum between `Xarray[i]` and `Yarray[i]`
- create an array `Zarray` of 1000 elements following an exp. distribution of param 0.3+0.1
- compare the histograms of `minXYarray` and of `Zarray`
***

In [ ]:
################################ TO COMPLETE
Xarray = ?????????????????
Yarray = ?????????????????
minXYarray = ??????????????
Zarray = ?????????????????



plt.hist(????????????????,normed=True,bins=30,alpha=.5,label='min(X,Y)')
plt.hist(????????????????,normed=True,bins=30,alpha=.5,label='Z')
plt.legend()
##################################

Let's check the second part (the "WHICH part") of the Property:

***Exercice:***
From the previously sampled arrays `Xarray` and `Yarray`, determine the proportion of times an element of `Xarray` happen before `Yarray`, compare this proportion to $\frac{0.1}{0.1+0.3}$


In [ ]:
################################ TO COMPLETE

?????????????????

###########################################

The previous property can be generalized to an abritrary number of exponentially distributed random variables:


***
**Property**: If you have $N$ events modelled by exponentially random variables whose respective rates are $a_1,a_2,\ldots,a_N$, then we have:
1. The time WHEN the first event happen follows a exponential distribution of parameter $a_1,a_2,\ldots,a_N$.
2. WHICH of the events happen first is given by a random weigthed choice with weights $$\left( \frac{a_1}{a_1+a_2+\ldots+a_N},\frac{a_2}{a_1+a_2+\ldots+a_N},\ldots,\frac{a_N}{a_1+a_2+\ldots+a_N}\right).$$ (that means that the probiblity that $i$-th event happen first is $\frac{a_i}{a_1+a_2+\ldots+a_N}$)
***

Why this property will be very usfull for us? Let's imagine we have many possible events (let's say 1000), and times of occurence os each event is modelled as a exp. random variables. How do we determine  which event happen first and when ? We have two equivalent ways:

- Method1 (naïve): we sample every 1000 exponentially random variables, we then determine which is the minimum
- Method2 (using the previous property): we sample 1 exponentially random variable (whose rate is the sum of all the rates) and we randomly picked which event depending on the weight of each event. To use this method, we need the function `randomWeightedChoice` previously defined. 

Note: In Method2 we will take advantage to the fact that `randomWeightedChoice` does not need normalized weights and directly return the sum of all the weights as a second output.

The Method2 is much more efficient as it is shown bellow:

In [ ]:
## Let's compare the perforance of the two methods:

ratesArray = np.random.random(size=1000) # this array will represent the rates of the 1000 random variables

def method1():
    events = numpy.zeros(1000)
    for i,r in enumerate(ratesArray): # for each rate
        events[i] = numpy.random.exponential(r) # let's sample a exp random variable according to the rate
    
    return events.argmin(), events.min()

def method2():    
    
    # We sample WHICH event will happen
    event,summ = randomWeightedChoice(ratesArray) # Let's use the function randomWeightedChoice we previously defined
    # event is the index that has been chosen
    # summ represents the sum of all the weigths (we did not need to normalize `ratesArray`, the function did it for us)
    
    # We sample WHEN this event will happen
    time_step = numpy.random.exponential(1./summ) 
    return event, time_step

# The strange following statements will test the two methods and compute how much time each of them takes.
%timeit method1()
%timeit method2()


# My first simulation: production and degradation of mRNAs

The Gillespeie Algorithm take advantage of the previously explained property to simulate stochastic models very efficiently and with any approximation. It follows the following steps:

***
**Algorithm**: Gillepsie Algorithm (Gillespie, 1977)

    initiation of the state of the simulation
    set time =0
    while time < T_lim :
        determine the rate of all possible events
        sample the next event and the time step for the next event to occur
        update time occording to the time step
        update the state according to the event sampled
***
        
Let's take an example : the production and degradation of mRNAs inside the cell. We can represent this using the following model:

Biology | Model
-|-
<img src="./figures/trans.svg" alt="Transcription" style="width: 200px;"/> |  <img src="./figures/TwoStageModel1.png" alt="TwoStageModel1.png" style="width: 200px;"/><br/>from Dessalles et al. (2017b)

The quantity $M$ will be the number of mRNA; it is the state of the simulation. There will be two possible events:
- the creation of mRNA. An event of mRNA creation will occur at exp. distributed times of rate $\lambda_1$. When that even occurs, we increase the number of mRNA $M$ by 1.
- the degradation of an mRNA. Each mRNA will degrade after an exp. distributed times of rate $\sigma_1$. With the previous property, the FIRST time at which ONE of all the mRNAs is degraded is given by a exp. distributed times of rate $\sigma_1M$. When that even occurs, we decrease the number of mRNA $M$ by 1.

***
**Exercice**: Let's implement this model using the Gillespie algorithm. The function `simulationTranscription1` takes different paremeters (see the comments), run the Gillespie algorithm and return the number of mRNAs at the end of the simulation. 
***


In [ ]:
def simulationOneStageModel1(T_lim=1000, lambda1 = 2, sigma1 = 1, M_init = 0):
    """
    Parameters:
    - T_lim: the time in minutes of how long is lasting the simulation
    - lambda1: the rate of mRNA creation
    - sigma1: the rate of mRNA degradation
    - M_init: the number of mRNA at the begining of the simulation
    
    Returns:
    - state: the state of the simulation after a simulation of T_lim
    """
    state =  M_init
    time = 0
    
    ################################ TO COMPLETE
    while .....................
    ?????????????
    ########################################
    
    return state

simulationOneStageModel1()

The previous function returns the number of mRNAs after $1000\,min$ (or whatever value you've choosen for T_lim). It is not that useful. Maybe would be interested seeing how mRNAs are evolving during the cell cycle.

For that, we need to regularly save the state of the simulation into a varible that we return at the end of the simulation. We propose to return two quantities:
- `saved_time` that will represent each time points at which the simulation has been saved (it depends on the time step `DeltaT` chosen as a parameter).
- `saved_state` that will represent what was the value of $M$ at each time point of `saved_time`

The algorithm will become:


***
**Algorithm**: Gillepsie Algorithm (Gillespie, 1977) (with saving)

    initialisation of the state of the simulation
    ==> initialisation of saving arrays
    set time =0
    while time < T_lim :
        determine the rate of all possible events
        sample the next event and the time step for the next event to occur
        update time occording to the time step
         ==> save the state in the saving arrays if needed
        update the state according to the event sampled
***

In [ ]:
def simulationOneStageModel2(T_lim=1000, lambda1 = 2, sigma1 = 1, M_init = 0, DeltaT = 1. ):
    """
    Parameters:
    - T_lim: the time in minute of how long is lasting the simulation
    - lambda1: the rate (min^-1) of mRNA creation
    - sigma1: the rate (min^-1) of mRNA degradation
    - M_init: the number of mRNA at the begining of the simulation
    - DeltaT: the time step (in min) at which the simulation is saved
    
    Returns:
    - saved_time: an array that represent each time point for which the state of the simulation has been saved
    - saved_state: an array that represent the state of the simulation at the corresponding point in saved_time
    """
    state =  M_init
    time = 0
    
    saved_time = np.arange(0,T_lim,DeltaT)
    saved_state = np.zeros(saved_time.shape)
    saved_state[0] = state
    index_saved_time = 1 # The index we will use to go through saved_time
    
    
    ################################ TO COMPLETE (you can begin by a copy paste of your previous function)
    while .....................
        ?????????????
        ??????????????????
         while index_saved_time<len(saved_time) and saved_time[index_saved_time]<= time:
            # if we need to save the state of the simulation in saved_state
            ???????????????
        ???????????????????
        ???????????????
    
    ########################################
    
    return saved_time,saved_state

saved_time,saved_state = simulationOneStageModel2()

plt.plot(saved_time,saved_state,label='mRNA number')
legend()
xlabel('Time [min]')
ylabel('Number')

Nicer, isn't ? Take time to play a little bit with the parameters to see what it changes

# A more complicated model: the two stage model

Let's then consider model a simple very simple model of protein production: the classical two-stage model (Paulsson, 2005).

Biology | Model
-|-
<img src="./figures/transtrad1.svg" alt="Transcription" style="width: 400px;"/>| <img src="./figures/TwoStageModel.png" alt="TwoStageModel.png" style="width: 400px;"/><br/>from Dessalles et al. (2017b)

Now, the state of the simulation is represented by two quantities:
- the number of mRNAs $M$
- the number of proteins $P$

In the simulation, I have chosen to represent the state by a dictionnary (of course, other solutions are possible).


And there is also additional events :
- the creation of mRNA at rate $\lambda_1$ (idem as for the previous model)
- the degradation of an mRNA at rate $\sigma_1M$ (idem as for the previous model)
- the creation of protein. Each mRNA will create a protein at an exp. distributed time of parameter $\lambda_2$. Thanks to the property of exponentially distribeted random variables, the FIRST time at which ONE protein is created is given by an exp. distributed time of rate $\lambda_2P$. When that even occurs, we increase the number of mRNA $P$ by 1.
- proteolysis. Each protein will degrade after an exp. distributed times of rate $\sigma_2$.So, the FIRST time at which ONE of all the proteins is degraded is given by a exp. distributed times of rate $\sigma_2P$. When that even occurs, we decrease the number of mRNA $P$ by 1.




In [ ]:
def simulationTwoStageModel(T_lim=1000, 
                        lambda1 = 2, sigma1 = 1, M_init = 0, 
                        lambda2 = 10, sigma2 = 0.011, P_init = 0, 
                        DeltaT = 1. ):
    """
    Parameters:
    - T_lim: the time in minute of how long will last the simulation
    - lambda1: the rate (min^-1) of mRNA creation
    - sigma1: the rate (min^-1) of mRNA degradation
    - M_init: the number of mRNA at the begining of the simulation
    - lambda2: the rate (min^-1) of protein creation per mRNA
    - sigma2: the rate (min^-1) of protein proteolysis
    - P_init: the number of mRNA at the begining of the simulation
    - DeltaT: the time step (in min) at which the simulation is saved
    
    Returns:
    - saved_time: an array that represent each time point for which the state of the simulation has been saved
    - saved_state: an array that represent the state of the simulation at the corresponding point in saved_time
    """
    state =  {'M':M_init,'P':P_init} # the state will be represented here as a dictionnary
    time = 0
    
    saved_time = np.arange(0,T_lim,DeltaT)
    saved_state = {'M':np.zeros(saved_time.shape),'P':np.zeros(saved_time.shape)}
    saved_state['M'][0] = state['M'] 
    saved_state['P'][0] = state['P'] 
    index_saved_time = 1 # The index we will use to go through saved_time
    
    ################################ TO COMPLETE (you can begin by a copy paste of your previous function)
    while .....................
    ?????????????
    ########################################
    
    return saved_time,saved_state

saved_time,saved_state = simulationTwoStageModel()

figure()
plt.plot(saved_time,saved_state['M'],label='mRNA number')
legend()
xlabel('Time [min]')
ylabel('Number of mRNA')
figure()
plt.plot(saved_time,saved_state['P'],color='r',label='protein number')
legend()
xlabel('Time [min]')
ylabel('Number of Proteins')



You can now begin the project with the information given at the begining of the Notebook.

# References

- Thisted, Thomas, and Kenn Gerdes. "Mechanism of post-segregational killing by the hok/sok system of plasmid R1: Sok antisense RNA regulates hok gene expression indirectly through the overlapping mok gene." Journal of molecular biology 223.1 (1992): 41-54.
- Franch, Thomas, Alexander P. Gultyaev, and Kenn Gerdes. "Programmed cell death by hok/sok of plasmid R1: Processing at the hok mRNA 3′-end triggers structural rearrangements that allow translation and antisense RNA binding11Edited by DE Draper." Journal of molecular biology 273, no. 1 (1997): 38-51.
- Gillespie, D.T. (1977). Exact stochastic simulation of coupled chemical reactions. J. Phys. Chem. 81, 2340–2361.
- Paulsson, J. (2005). Models of stochastic gene expression. Physics of Life Reviews 2, 157–175.
- Dessalles, R., Fromion, V., and Robert, P. (2017a). A stochastic analysis of autoregulation of gene expression. J. Math. Biol. 1–31.
- Dessalles, R., Fromion, V., and Robert, P. (2017b). Models of protein production with cell cycle. ArXiv:1711.06378 [q-Bio].


